In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from utils import *
from docplex.mp.model import Model

In [13]:
fname = "..\\..\\data\\NRP\\NRC\\Instance6.txt"
problem = NRCProblem()
problem.load_from_txt(fname)


In [14]:
problem.save_to_json("..\\..\\data\\NRP\\NRC\\Instance6.json")

In [15]:
problem2 = NRCProblem()
problem2.load_from_json("..\\..\\data\\NRP\\NRC\\Instance6.json")


In [ ]:
# TODO: idea with sequences

# all_shifts = all_shifts + ['VACATION']
#
# assigned = [mdl.sequence_var([shift for shift in all_shifts], types=[shift for shift in all_shifts], name="nurse_{}".format(nurse)) for nurse in all_nurses]

# print("assigned: ", assigned)

In [16]:
mdl = Model(name="nurses")

# first global collections to iterate upon
all_nurses = problem.staff.keys()
all_shifts = problem.shifts.keys()
all_days = list(range(problem.horizon))

# the assignment variables.
assigned = mdl.binary_var_cube(keys1=all_nurses, keys2=all_days, keys3=all_shifts, name="assign_%s_%s_%s")
print("assigned: ", assigned)

assigned:  {('A', 0, 'E'): docplex.mp.Var(type=B,name='assign_A_0_E'), ('A', 0, 'D'): docplex.mp.Var(type=B,name='assign_A_0_D'), ('A', 0, 'L'): docplex.mp.Var(type=B,name='assign_A_0_L'), ('A', 1, 'E'): docplex.mp.Var(type=B,name='assign_A_1_E'), ('A', 1, 'D'): docplex.mp.Var(type=B,name='assign_A_1_D'), ('A', 1, 'L'): docplex.mp.Var(type=B,name='assign_A_1_L'), ('A', 2, 'E'): docplex.mp.Var(type=B,name='assign_A_2_E'), ('A', 2, 'D'): docplex.mp.Var(type=B,name='assign_A_2_D'), ('A', 2, 'L'): docplex.mp.Var(type=B,name='assign_A_2_L'), ('A', 3, 'E'): docplex.mp.Var(type=B,name='assign_A_3_E'), ('A', 3, 'D'): docplex.mp.Var(type=B,name='assign_A_3_D'), ('A', 3, 'L'): docplex.mp.Var(type=B,name='assign_A_3_L'), ('A', 4, 'E'): docplex.mp.Var(type=B,name='assign_A_4_E'), ('A', 4, 'D'): docplex.mp.Var(type=B,name='assign_A_4_D'), ('A', 4, 'L'): docplex.mp.Var(type=B,name='assign_A_4_L'), ('A', 5, 'E'): docplex.mp.Var(type=B,name='assign_A_5_E'), ('A', 5, 'D'): docplex.mp.Var(type=B,name='a

In [17]:
# vacations

vacations = []

for nurse in problem.days_off.keys():
    for day in problem.days_off[nurse]:
        # print(nurse, day)
        for shift in problem.cover_requirements:
            if shift['Day'] == day:
                temp = assigned[nurse, shift['Day'], shift['ShiftID']]
                vacations.append(temp)
                mdl.add_constraint(temp == 0)
        # for day in days:
        #     vacations.append(assigned[nurse, day])

print("vacations: ", vacations)

vacations:  [docplex.mp.Var(type=B,name='assign_A_18_E'), docplex.mp.Var(type=B,name='assign_A_18_D'), docplex.mp.Var(type=B,name='assign_A_18_L'), docplex.mp.Var(type=B,name='assign_A_19_E'), docplex.mp.Var(type=B,name='assign_A_19_D'), docplex.mp.Var(type=B,name='assign_A_19_L'), docplex.mp.Var(type=B,name='assign_B_24_E'), docplex.mp.Var(type=B,name='assign_B_24_D'), docplex.mp.Var(type=B,name='assign_B_24_L'), docplex.mp.Var(type=B,name='assign_B_25_E'), docplex.mp.Var(type=B,name='assign_B_25_D'), docplex.mp.Var(type=B,name='assign_B_25_L'), docplex.mp.Var(type=B,name='assign_C_22_E'), docplex.mp.Var(type=B,name='assign_C_22_D'), docplex.mp.Var(type=B,name='assign_C_22_L'), docplex.mp.Var(type=B,name='assign_C_23_E'), docplex.mp.Var(type=B,name='assign_C_23_D'), docplex.mp.Var(type=B,name='assign_C_23_L'), docplex.mp.Var(type=B,name='assign_D_4_E'), docplex.mp.Var(type=B,name='assign_D_4_D'), docplex.mp.Var(type=B,name='assign_D_4_L'), docplex.mp.Var(type=B,name='assign_D_5_E'), d

In [18]:
# max 1 shift per day

for nurse in all_nurses:
    for day in all_days:
        mdl.add_constraint(mdl.sum(assigned[nurse, day, shift] for shift in all_shifts) <= 1)

In [21]:
# eliminate following shifts

for nurse in all_nurses:
    for day in all_days[:-1]:
        for shift in all_shifts:
            forbidden = problem.shifts[shift]['CannotFollow']
            # print(forbidden)
            if len(forbidden) > 0:
                mdl.add_constraint(assigned[nurse, day, shift] + mdl.sum(assigned[nurse, day + 1, f] for f in forbidden) <= 1)


In [ ]:
# The maximum numbers of shifts of each type that can be assigned to employees.

for nurse in all_nurses:
    for shift in all_shifts:
        mdl.add_constraint(mdl.sum(assigned[nurse, day, shift] for day in all_days) <= problem.staff[nurse][shift])

In [ ]:
# Minimum and maximum work time

for nurse in all_nurses:
    mdl.add_constraint(mdl.sum(assigned[nurse, day, shift] * problem.shifts[shift]['Duration'] for day in all_days for shift in all_shifts) >= problem.staff[nurse]['MinWorkTime'])
    mdl.add_constraint(mdl.sum(assigned[nurse, day, shift] * problem.shifts[shift]['Duration'] for day in all_days for shift in all_shifts) <= problem.staff[nurse]['MaxWorkTime'])

In [24]:
# Maximum consecutive shifts. The maximum number of shifts an employee can work without a day off.
# Each nurse can work at most 'max_consecutive_shifts' consecutive days.

for nurse in all_nurses:
    max_consecutive_shifts = problem.staff[nurse]['max_consecutive_shifts']
    for day in all_days[:-max_consecutive_shifts]:
        mdl.add_constraint(mdl.sum(assigned[nurse, day + i, shift] for i in range(max_consecutive_shifts) for shift in all_shifts) <= max_consecutive_shifts)


In [ ]:
# Minimum consecutive shifts.

for nurse in all_nurses:
    min_consecutive_shifts = problem.staff[nurse]['min_consecutive_shifts']
    for day in all_days[:-min_consecutive_shifts]:
        mdl.add_constraint(mdl.sum(assigned[nurse, day + i, shift] for i in range(min_consecutive_shifts) for shift in all_shifts) >= min_consecutive_shifts)

In [ ]:
# Minimum consecutive days off

for nurse in all_nurses:
    min_consecutive_days_off = problem.staff[nurse]['min_consecutive_days_off']
    for day in all_days[:-min_consecutive_days_off]:
        mdl.add_constraint(mdl.sum(not assigned[nurse, day + i, shift] for i in range(min_consecutive_days_off) for shift in all_shifts) == 0)
